In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from model2 import enhance_net

In [ ]:
def NormalizeData(data):
    return (data - tf.math.reduce_min(data)) / (tf.math.reduce_max(data) - tf.math.reduce_min(data))

In [ ]:
def image_selection(enhancement_images):
    images = []
    for index, i in enumerate(enhancement_images):
        r,g,b = tf.split(i, 3, axis=2)
        r_g = int(np.abs(np.mean(r*0.299-g*0.587)))
        r_b = int(np.abs(np.mean(r*0.299-b*0.114)))
        g_b = int(np.abs(np.mean(g*0.587-b*0.114)))
        
        img = tf.image.rgb_to_hsv(i)
        h,s,v = tf.split(img, 3, axis=2)
        mean_hue = tf.math.reduce_mean(h*255)
        mean_v = tf.math.reduce_mean(v)
        
        if 22 < mean_hue and mean_hue <= 100:
            if 30 <= mean_v and mean_v <= 145:
                if  5<=r_g and  r_g <= 36: 
                    if 5 <= r_b and r_b<=48:
                        if 5 <= g_b and  g_b <= 75:
                            images.append(i)
                            print(index)
    if np.shape(images)[0] == 0:
        return enhancement_images[0:5]
    return images 

In [ ]:
model_name = ['DCE_Denoised_noactivate']
model_name = model_name[0]
add_noise = True
print(model_name)

In [ ]:
i = 'finish'
enhancement_path = './model/{0}/enhancment/weights/{1}/'.format(model_name, i)
# denoising_path = './model/{0}/denoising/weights/epoch{1}/'.format(model_name, i)
model = enhance_net(input_shape=(None,None,3), model_name=model_name, train=False, add_noise=add_noise)
model.enhancement_net.load_weights(enhancement_path)
# model.denoising_net.load_weights(denoising_path)

img_dir = './Dataset/test2/'
output_dir = './result/{0}/{1}/'.format(model_name, i)

folder = os.path.exists(output_dir)
if not folder:
    os.makedirs(output_dir)

for file in os.listdir(img_dir):
    img_path = img_dir + file
    img = cv2.imread(img_path)
    cv2.imwrite(output_dir+file, img)
    
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)/255
    h,w,c = np.shape(img)
    img = tf.reshape(img, (1,h,w,c))
    pred_img = tf.cast(img, dtype=tf.float32)
    
    

    parameter_map, noise, denoise = model.predict(pred_img)
    
    pred_img = denoise
    
    # pred_img = tf.cast((denoise) * 255, dtype=tf.int8)
    # print(tf.reduce_min(pred_img))
    # pred_img = tf.cast(pred_img/255, dtype=tf.float32)
    
        
    denoise = np.reshape(denoise, (h,w,c))
    denoise = cv2.cvtColor(denoise, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_dir + file[:-4] + 'denoise.jpg', denoise*255)
    
    noise = np.reshape(noise, (h,w,c))
    noise = cv2.cvtColor(noise, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_dir + file[:-4] + 'noise.jpg', noise*255)
    
    
    
    # ----------------對low-light Image增強----------------
    enhancement_images = []
    for i in range(8):
        pred_img = pred_img + parameter_map * (pred_img - tf.math.pow(pred_img, 2))
        pred_img = NormalizeData(pred_img)
        enhancement_image = np.reshape(pred_img, (h,w,c))*255
        # cv2.imwrite("./image/test/{0}.jpg".format(i+1), cv2.cvtColor(np.asarray(enhancement_image), cv2.COLOR_RGB2BGR))
        enhancement_images.append(enhancement_image)
    # -----------------------end--------------------------
    
    parameter_map = np.reshape(parameter_map, (h,w,c))
    parameter_map = cv2.cvtColor(parameter_map, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_dir + file[:-4] + 'parameter_map.jpg', parameter_map*255)
    
    # ------------------------圖像挑選--------------------
    # enhancement_images = image_selection(enhancement_images)
    # print(file)
    # print(np.shape(enhancement_images))
    # ----------------------------------------------------
    
    
    # ------------Merge using Exposure Fusion-------------
    # print("Merging using Exposure Fusion ... ")
    mergeMertens = cv2.createMergeMertens()
    exposureFusion = mergeMertens.process(enhancement_images[:5])
    # plt.imshow(exposureFusion)
    # plt.show()

    # Save output image
    # print("Saving output ... exposure-fusion.jpg")
    exposureFusion = cv2.cvtColor(exposureFusion,cv2.COLOR_BGR2RGB)
    cv2.imwrite(output_dir + file[:-4]+'exposure-fusion.png', exposureFusion*255)
    # -----------------------end--------------------------